In [1]:
# Install the required libraries
!pip install prophet
!pip install plotly
!pip install pmdarima

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 63.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 69.0 MB/s eta 0:00:00


In [2]:
# Import the required libraries and dependencies
import pandas as pd
import datetime as dt
from prophet import Prophet
import  matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
%matplotlib inline

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Read csv file as DataFrame, with the first column as row index
data = pd.read_csv("./Resources/credit_card_transactions.csv")

FileNotFoundError: [Errno 2] No such file or directory: './Resources/credit_card_transactions.csv'

In [6]:
# Create a df where the year, month, day are made into their own columns along with just the trans_date
df = data.copy()
df["trans_year"] = list(map(lambda x: x.split(" ")[0].split("-")[0], df["trans_date_trans_time"]))
df["trans_month"] = list(map(lambda x: x.split(" ")[0].split("-")[1], df["trans_date_trans_time"]))
df["trans_day"] = list(map(lambda x: x.split(" ")[0].split("-")[2], df["trans_date_trans_time"]))
df["trans_date"] = list(map(lambda x: x.split(" ")[0], df["trans_date_trans_time"]))

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,trans_num,unix_time,merch_lat,merch_long,is_fraud,merch_zipcode,trans_year,trans_month,trans_day,trans_date
1296670,1296670,2020-06-21 12:12:08,30263540414123,fraud_Reichel Inc,entertainment,15.56,Erik,Patterson,M,162 Jessica Row Apt. 072,...,440b587732da4dc1a6395aba5fb41669,1371816728,36.841266,-111.690765,0,NaN,2020,06,21,2020-06-21
1296671,1296671,2020-06-21 12:12:19,6011149206456997,fraud_Abernathy and Sons,food_dining,51.70,Jeffrey,White,M,8617 Holmes Terrace Suite 651,...,278000d2e0d2277d1de2f890067dcc0a,1371816739,38.906881,-78.246528,0,22630.0,2020,06,21,2020-06-21
1296672,1296672,2020-06-21 12:12:32,3514865930894695,fraud_Stiedemann Ltd,food_dining,105.93,Christopher,Castaneda,M,1632 Cohen Drive Suite 639,...,483f52fe67fabef353d552c1e662974c,1371816752,33.619513,-105.130529,0,88351.0,2020,06,21,2020-06-21
1296673,1296673,2020-06-21 12:13:36,2720012583106919,"fraud_Reinger, Weissnat and Strosin",food_dining,74.90,Joseph,Murray,M,42933 Ryan Underpass,...,d667cdcbadaaed3da3f4020e83591c83,1371816816,42.788940,-103.241160,0,69367.0,2020,06,21,2020-06-21
1296674,1296674,2020-06-21 12:13:37,4292902571056973207,"fraud_Langosh, Wintheiser and Hyatt",food_dining,4.30,Jeffrey,Smith,M,135 Joseph Mountains,...,8f7c8e4ab7f25875d753b422917c98c9,1371816817,46.565983,-114.186110,0,59870.0,2020,06,21,2020-06-21


In [94]:
# Create a df_merged where it has the ratio of transactions, ratio of total spend, and ratio of transactions in the last 90 days. This is done to see the
# cc number's contribution to the total transaction amount, total spend, and total transactions in the last 90 days.
cc_nums = df["cc_num"].unique()
numofdays = (pd.to_datetime(df["trans_date_trans_time"].iloc[-1])-pd.to_datetime(df["trans_date_trans_time"].iloc[0])).days
numoftrans = df["Unnamed: 0"].count()
numofrecenttrans = df[df["trans_date_trans_time"]>"2020-03-21"]["Unnamed: 0"].count()

df_merged = pd.concat([df.groupby("cc_num").agg(totaltransactions = ("amt","count"))/numoftrans,
                        df.groupby("cc_num").agg(totalspend = ("amt","sum"))/df["amt"].sum(),
                        df[df["trans_date_trans_time"]>="2020-03-21"].groupby("cc_num").agg(numberofrecenttransactions = ("amt","count"))/numofrecenttrans],
                        axis=1)
df_merged.rename(columns={"totaltransactions": "Total Transactions Ratio", "totalspend":"Total Spend Ratio","numberofrecenttransactions":"Total Recent Transactions Ratio (last 90 days)"},inplace=True)


In [95]:
# Creates a ltv_df using the df_merged and fills in the NAN values with 0
ltv_df = df_merged.fillna(0)
# Creates a LTV Score column where a LTV score is calculated for every cc.
# Weighted by 25% on the total transaction ratio a cc number contributed, 50% on the total spend ratio a cc number contributed,
# and 25% on the total transaction ratio a cc number contributed in the last 90 days.
ltv_df["LTV Score"] = ltv_df["Total Transactions Ratio"] * .25 + ltv_df["Total Spend Ratio"] * .50 + ltv_df["Total Recent Transactions Ratio (last 90 days)"] * .25
# Sort the dataframe based on LTV score.
ltv_df.sort_values(by="LTV Score", ascending=False, inplace=True)
# Normalizes the data based on the top LTV score. This gives the highest LTV score a score of 100.
ltv_df["LTV Score"] = ltv_df["LTV Score"] / ltv_df["LTV Score"].max() * 100

,Total Transactions Ratio,Total Spend Ratio,Total Recent Transactions Ratio (last 90 days),LTV Score
cc_num,,,,
6011367958204270,0.002398,0.003250,0.002441,100.000000
4908846471916297,0.002344,0.003184,0.002357,97.622636
36722699017270,0.002405,0.003070,0.002459,97.040909
4836998673805450,0.002334,0.003025,0.002596,96.838059
3590736522064285,0.002376,0.002994,0.002614,96.826349
...,...,...,...,...
375623337201129,0.000008,0.000036,0.000000,0.700875
38817501916673,0.000006,0.000033,0.000000,0.639006
4734310647841293,0.000005,0.000027,0.000000,0.529890


In [96]:
# Calculate the 75% LTV score cutoff.
top25percentltv = ltv_df["LTV Score"].describe().iloc[6]
# Create a list of cc nubmers that contains the cc numbers with a LTV score higher than the 75% cutoff
ltv_top25percent = ltv_df[ltv_df["LTV Score"]>=top25percentltv]
# Creates a df with all the info of the top 25% cc numbers by LTV scores.
topltvcustomers = df.loc[df["cc_num"].isin(ltv_top25percent.index)]

,Total Transactions Ratio,Total Spend Ratio,Total Recent Transactions Ratio (last 90 days),LTV Score
cc_num,,,,
6011367958204270,0.002398,0.003250,0.002441,100.000000
4908846471916297,0.002344,0.003184,0.002357,97.622636
36722699017270,0.002405,0.003070,0.002459,97.040909
4836998673805450,0.002334,0.003025,0.002596,96.838059
3590736522064285,0.002376,0.002994,0.002614,96.826349
...,...,...,...,...
3598304579370089,0.001572,0.001345,0.001520,50.989573
3583793405872580,0.001562,0.001236,0.001737,50.899009
4128730454058057622,0.001569,0.001335,0.001515,50.736040


In [115]:
# Create a df for total spend per day
totalspendperday = df.groupby("trans_date").agg(TotalSpent = ("amt","sum"))
# Create a df for a the total spend per day of the top 25% cc numbers by ltv score
topltvcustomers_groupby = topltvcustomers.groupby("trans_date").agg(totalspend = ("amt","sum"))
# Create a figure on these two data frames to see how much money the top 25% of cc's are contributing. (about 46%)
fig = go.Figure()
fig.add_trace(go.Scatter(x=totalspendperday.index,y=totalspendperday["TotalSpent"],mode="lines",name="All Customers",line=dict(color="blue",width=2)))
fig.add_trace(go.Scatter(x=topltvcustomers_groupby.index,y=topltvcustomers_groupby["totalspend"],mode="lines",name="Top 25% LTV Customers",line=dict(color="orange",width=2)))
fig.update_layout(title="All Customers vs. Top 25% LTV Customers Total Spending per Day",xaxis_title="Date",yaxis_title="Amount Spent ($)",showlegend=True)
fig.show()

In [128]:
# Create a 3 year forecast for the top 25% of cc's by ltv score using Prophet
prophet_df = topltvcustomers_groupby.reset_index()

prophet_df.columns = ['ds', 'y']
prophet_df.head()

m = Prophet()

m.fit(prophet_df)
future = m.make_future_dataframe(periods=365*3)
forecast = m.predict(future)

forecast_df = pd.DataFrame(forecast["yhat"])
forecast_df.index = pd.date_range(start="2019-01-01",end="2023-06-20")

forecast_df_lower = pd.DataFrame(forecast["yhat_lower"])
forecast_df_lower.index = pd.date_range(start="2019-01-01",end="2023-06-20")

forecast_df_upper = pd.DataFrame(forecast["yhat_upper"])
forecast_df_upper.index = pd.date_range(start="2019-01-01",end="2023-06-20")


fig = go.Figure()
fig.add_trace(go.Scatter(x=topltvcustomers_groupby.index,y=topltvcustomers_groupby["totalspend"],mode="markers",name="Top 25% LTV Customers",marker=dict(symbol="circle",color="blue",size=5)))
fig.add_trace(go.Scatter(x=forecast_df.index,y=forecast_df["yhat"],mode="lines",name="Prediction",line=dict(color="orange",width=2)))
fig.add_trace(go.Scatter(x=forecast_df_lower.index,y=forecast_df_lower["yhat_lower"],mode="lines",name="Prediction Lower",line=dict(color="red",width=.25)))
fig.add_trace(go.Scatter(x=forecast_df_upper.index,y=forecast_df_upper["yhat_upper"],mode="lines",name="Prediction Upper",line=dict(color="green",width=.25)))

fig.update_layout(title="Prophet Forecast of Top 25% LTV Customers",xaxis_title="Date",yaxis_title="Amount Spent ($)",showlegend=True)
fig.update_yaxes(range=[0, 250000])

fig.show()

00:18:12 - cmdstanpy - INFO - Chain [1] start processing
00:18:12 - cmdstanpy - INFO - Chain [1] done processing


In [120]:
# Check if data is stationary
dftest = adfuller(topltvcustomers_groupby["totalspend"], autolag="AIC")
# The result is 0.00859, the lower this number the better
print(dftest[1])

from pmdarima import auto_arima

import warnings
warnings.filterwarnings("ignore")

stepwise_fit = auto_arima(topltvcustomers_groupby["totalspend"],trace=True,suppress_warnings=True)
# Determine what ARIMA values is best to fit this dataset
stepwise_fit.summary()

0.008590815411667015


In [124]:
# Train the ARIMA model, fit it, and make a 1 year prediction
train=topltvcustomers_groupby.iloc[:]
model=ARIMA(train["totalspend"],order=(2,1,3))
model=model.fit()
model.summary()
start=1
end=538
pred=model.predict(start=start,end=end,typ="levels")
pred.index=pd.date_range(start="2020-01-01",end="2021-06-21")

In [127]:
# Create one dataframe containing the top 25% cc's by LTV score total spend per day and append the predictions at the end.
ss = pd.concat([topltvcustomers_groupby,pred])
ss.rename(columns={"predicted_mean":"predictedspend"},inplace=True)

fig = go.Figure()
fig.add_trace(go.Scatter(x=ss.index,y=ss["totalspend"],mode="lines",name="Top 25% LTV Customers",line=dict(color="blue",width=2)))
fig.add_trace(go.Scatter(x=ss.index,y=ss["predictedspend"],mode="lines",name="Prediction",line=dict(color="orange",width=2)))

fig.update_layout(title="ARIMA Forecast of Top 25% LTV Customers",xaxis_title="Date",yaxis_title="Amount Spent ($)",showlegend=True)
fig.update_yaxes(range=[0, 250000])